In [35]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, RocCurveDisplay

from tqdm import tqdm

import optuna

In [36]:
data = np.load('processed_ntuples\chunk_data.npy')
np.shape(data)

(529663, 33)

In [37]:
train, val = train_test_split(data, random_state=137) # setting the seed for reproducibility; the default train/test split is 0.75/0.25
print('lenght of training set', len(train), '\nlenght of validation set', len(val), '\nwidth of the sets: # of features + 1 flag (1-LL/0-TX)', len(train[0]))

lenght of training set 397247 
lenght of validation set 132416 
width of the sets: # of features + 1 flag (1-LL/0-TX) 33


In [38]:
# strip the last feature (e.g. the flag) from the sets

# copy in dedicated arrays
flags_train = train[:, -1]
flags_val = val[:, -1]
# delete from the sets
train = train[:, :-1]
val = val[:, :-1]
# width check
print(len(train[0]), len(val[0]))

32 32


In [39]:
# tensorize the data, so that pytorch doesn't whine
train = torch.tensor(train, dtype=torch.float32)
val = torch.tensor(val, dtype=torch.float32)
flags_train = torch.tensor(flags_train, dtype=torch.float32)
flags_val = torch.tensor(flags_val, dtype=torch.float32)

In [40]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0.01):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_score = None

    def early_stop(self, validation_loss):
        score = -validation_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.best_score= validation_loss
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.best_score = score
            self.counter = 0
        return False

In [41]:
# 1. Define an objective function to be maximized.
def objective(trial):

    n_epochs = 50

    loss_function = nn.BCELoss() # CrossEntropyLoss but for just one class

    # Early stopping
    early_stopping = EarlyStopping(patience=5, delta=0.01)

    # 2. Suggest values of the hyperparameters using a trial object.
    n_layers = trial.suggest_int('n_layers', 2, 4)
    layers = []

    batch_size = trial.suggest_int('batch_size', 50, 5000)
    batch_start = torch.arange(0, len(train), batch_size)

    in_features = len(train[0])
    for i in range(n_layers):
        out_features = trial.suggest_int(f'n_units_l{i}', 40, 100)
        layers.append(torch.nn.Linear(in_features, out_features))
        layers.append(nn.BatchNorm1d(out_features))
        layers.append(torch.nn.ReLU())
        layers.append(torch.nn.Dropout(0.25))
        in_features = out_features
    layers.append(torch.nn.Linear(in_features, 1))
    layers.append(nn.Sigmoid())
    model = torch.nn.Sequential(*layers).to(torch.device('cpu'))

    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(n_epochs):
        model.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                train_batch = train[start:start+batch_size]
                flags_train_batch = flags_train[start:start+batch_size]
                # forward pass
                outputs = model(train_batch)
                loss = loss_function(outputs, flags_train_batch.unsqueeze(0).T)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (outputs.round() == flags_train_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )

        model.eval()
        with torch.no_grad():
            val_outputs = model(val)
            val_loss = loss_function(val_outputs, flags_val.unsqueeze(0).T)
            accuracy = float((val_outputs.round().T == flags_val).float().mean())

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
        # Early Stopping
        if early_stopping.early_stop(val_loss):
            print(f'Early stopped at epoch: {epoch}')
            break

    early_stopping = EarlyStopping(patience=5, delta=0.01)

    return accuracy


# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-03-06 22:08:02,759] A new study created in memory with name: no-name-85ba4237-9c1f-4ea0-8487-30f91ae07771


[I 2025-03-06 22:08:55,198] Trial 0 finished with value: 0.7140300273895264 and parameters: {'n_layers': 3, 'batch_size': 2953, 'n_units_l0': 72, 'n_units_l1': 68, 'n_units_l2': 41, 'lr': 0.00013109046372193687}. Best is trial 0 with value: 0.7140300273895264.


Early stopped at epoch: 6


[I 2025-03-06 22:09:41,863] Trial 1 finished with value: 0.7483989596366882 and parameters: {'n_layers': 2, 'batch_size': 1634, 'n_units_l0': 82, 'n_units_l1': 88, 'lr': 0.0009492902754184888}. Best is trial 1 with value: 0.7483989596366882.


Early stopped at epoch: 7


...

[I 2025-03-06 22:48:43,831] Trial 38 pruned. 
[I 2025-03-06 22:48:53,842] Trial 39 pruned. 
[I 2025-03-06 22:49:58,430] Trial 40 pruned. 
[I 2025-03-06 22:51:00,916] Trial 41 pruned. 
[I 2025-03-06 22:52:01,223] Trial 42 pruned. 
[I 2025-03-06 22:53:28,768] Trial 43 pruned. 
[I 2025-03-06 22:53:37,765] Trial 44 pruned. 
[I 2025-03-06 22:54:37,306] Trial 45 pruned. 
[I 2025-03-06 22:54:45,040] Trial 46 pruned. 
[I 2025-03-06 22:54:53,095] Trial 47 pruned. 
[I 2025-03-06 22:55:04,588] Trial 48 pruned. 
[I 2025-03-06 22:55:10,383] Trial 49 pruned. 
[I 2025-03-06 22:56:23,251] Trial 50 pruned. 
[I 2025-03-06 22:56:47,858] Trial 51 pruned. 
[I 2025-03-06 22:57:24,278] Trial 52 pruned. 
[I 2025-03-06 22:57:31,154] Trial 53 pruned. 
[I 2025-03-06 22:58:40,998] Trial 54 pruned. 
[I 2025-03-06 22:59:42,283] Trial 55 pruned. 
[I 2025-03-06 22:59:48,418] Trial 56 pruned. 
[I 2025-03-06 23:00:46,980] Trial 57 pruned. 
[I 2025-03-06 23:00:55,651] Trial 58 pruned. 
[I 2025-03-06 23:02:01,040] Trial 

Early stopped at epoch: 14


[I 2025-03-06 23:05:04,802] Trial 64 pruned. 
[I 2025-03-06 23:06:08,132] Trial 65 pruned. 
[I 2025-03-06 23:07:23,768] Trial 66 pruned. 
[I 2025-03-06 23:07:45,477] Trial 67 pruned. 
[I 2025-03-06 23:07:56,252] Trial 68 pruned. 
[I 2025-03-06 23:08:05,961] Trial 69 pruned. 
[I 2025-03-06 23:08:38,848] Trial 70 pruned. 
[I 2025-03-06 23:08:45,833] Trial 71 pruned. 
[I 2025-03-06 23:09:31,843] Trial 72 pruned. 
[I 2025-03-06 23:09:38,241] Trial 73 pruned. 
[I 2025-03-06 23:10:04,189] Trial 74 pruned. 
[I 2025-03-06 23:10:43,398] Trial 75 pruned. 
[I 2025-03-06 23:10:49,717] Trial 76 pruned. 
[I 2025-03-06 23:10:56,080] Trial 77 pruned. 
[I 2025-03-06 23:11:03,875] Trial 78 pruned. 
[I 2025-03-06 23:12:10,836] Trial 79 pruned. 
[I 2025-03-06 23:12:22,544] Trial 80 pruned. 
[I 2025-03-06 23:13:11,159] Trial 81 pruned. 
[I 2025-03-06 23:13:49,080] Trial 82 pruned. 
[I 2025-03-06 23:14:06,365] Trial 83 pruned. 
[I 2025-03-06 23:14:16,014] Trial 84 pruned. 
[I 2025-03-06 23:15:15,439] Trial 

Early stopped at epoch: 7


[I 2025-03-06 23:21:30,887] Trial 99 pruned. 


In [42]:
print("Best trial:")
b_trial = study.best_trial
print("  Value: ", b_trial.value)

print("  Params: ")
for key, value in b_trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  0.8652051091194153
  Params: 
    n_layers: 2
    batch_size: 2036
    n_units_l0: 98
    n_units_l1: 91
    lr: 0.00965812692572207


In [43]:
file = open('optuna_res_BS.txt', 'w')

for key, value in b_trial.params.items():
    print("    {}: {}".format(key, value))
    file.write("    {}: {}\n".format(key, value))

file.close()

    n_layers: 2
    batch_size: 2036
    n_units_l0: 98
    n_units_l1: 91
    lr: 0.00965812692572207


In [44]:
#